# Optimisation of analytical functions

In [1]:
// include header files

#include <iostream>

#include "xtensor/xarray.hpp"
#include "xtensor/xnpy.hpp"
#include "xtensor/xio.hpp"

#include "xevo/pso.hpp"
#include "xevo/functors.hpp"
#include "xevo/analytical_functions.hpp"

## Rosenbrock function

Optimise Rosenbrock function with `xevo::pso`.

Rosenbrock function is expressed as:

$$
 f(x_1, x_2) = 100(x_1^2 - x_2) + (1 - x_1)^2 \quad with \quad \bf{X} \quad \in \left[-3, 3\right]
$$

In [2]:
{
  using xtensor_x_type = xt::xarray<double>;
  using objective_type = xevo::Rosenbrock;
  using population_type = xevo::Population;
  using selection_type = xevo::Selection_best_pso;
  using velocity_type = xevo::Velocity_ring_topology;
  using position_type = xevo::Position;

  std::array<std::size_t, 2> shape = {40, 2};
  std::array<std::size_t, 1> shape_y = {40};

  xt::xarray<double> X = xt::zeros<double>(shape);
  xt::xarray<double> V = xt::zeros<double>(shape);
  double max_double_value = std::numeric_limits<double>::max();
  
  xevo::Rosenbrock objective_f;

  xevo::pso pso_algorithm;
  pso_algorithm.initialise<xtensor_x_type,
   population_type, population_type>(X, V);
  
  xt::xarray<double> XB(X);
  xt::xarray<double> YB = xt::ones<double>(shape_y)*max_double_value;

  std::size_t num_generations = 200;
  for (auto i{0}; i<num_generations; ++i)
  {
    pso_algorithm.evolve<xtensor_x_type, xtensor_x_type, objective_type,
     position_type, velocity_type, selection_type>(X, XB, YB, V, objective_f, std::make_tuple(),
     std::make_tuple(0.5, 0.8, 0.9), std::make_tuple());
  }


std::cout << "Last pop: \n" << X << "\n" << std::endl;

}

Last pop: 
{{ 0.666993,  0.667319},
 { 0.666447,  0.666256},
 { 0.666525,  0.666377},
 { 0.666519,  0.666371},
 { 0.666519,  0.666371},
 { 0.666519,  0.666371},
 { 0.666544,  0.666418},
 { 0.66655 ,  0.666435},
 { 0.666549,  0.666432},
 { 0.666613,  0.66655 },
 { 0.666719,  0.666773},
 { 0.666659,  0.666652},
 { 0.666662,  0.666658},
 { 0.666665,  0.666664},
 { 0.666665,  0.666664},
 { 0.666665,  0.666664},
 { 0.666666,  0.666665},
 { 0.666666,  0.666665},
 { 0.666667,  0.666667},
 { 0.666667,  0.666667},
 { 0.666666,  0.666665},
 { 0.666666,  0.666666},
 { 0.666667,  0.666667},
 { 0.666666,  0.666666},
 { 0.666667,  0.666668},
 { 0.666668,  0.666669},
 { 0.666667,  0.666666},
 { 0.666667,  0.666666},
 { 0.666667,  0.666666},
 { 0.666666,  0.666666},
 { 0.666666,  0.666665},
 { 0.666669,  0.666671},
 { 0.666669,  0.666671},
 { 0.666663,  0.66666 },
 { 0.666665,  0.666663},
 { 0.66665 ,  0.666633},
 { 0.666632,  0.666598},
 { 0.666955,  0.667242},
 { 0.666719,  0.666771},
 { 0.666679,  

## Branin function

Optimise Branin function with `xevo::pso`.

Branin function is expressed as:

$$
 f(x) = ( x_2 - \frac{5.1}{4\pi^2}x_2 + \frac{5}{\pi}x_1 - 6 )^2 + 10\left[ (1 - \frac{1}{8\pi})\cos{x_1} + 1 \right] + 5x_1\\
				with \quad x_1 \in \left[-5,10\right], x_2 \in \left[0,15\right] 
$$

In [3]:
struct Branin
{

/**
 * @brief operator to evaluate the objective function.
 * 
 * @tparam E xtensor type
 * @tparam value_type xtensor value type 
 * @param X array to be evaluated
 * @return auto evaluated array
 */
template <class E, typename value_type = typename std::decay_t<E>::value_type>
inline auto operator()(const xt::xexpression<E>& X)
{
  const E& _X = X.derived_cast();

  auto shape = _X.shape();
  std::size_t dim = _X.dimension();
  if (dim != 2)
  {
    throw std::runtime_error("The input array should be of dim 2");
  }
  xt::xtensor<value_type, 1, xt::layout_type::row_major> _x1(xt::view(_X, xt::all(), 0));
  xt::xtensor<value_type, 1, xt::layout_type::row_major> _x2(xt::view(_X, xt::all(), 1));

  auto X1 = 15 * _x1 - 5;
  auto X2 = 15 * _x2;
  value_type a = 1;
  value_type b = 5.1 / (4 * xt::numeric_constants<value_type>::PI*
    xt::numeric_constants<value_type>::PI);
  value_type c = 5 / xt::numeric_constants<value_type>::PI;
  value_type d = 6;
  value_type e = 10;
  value_type f = 1 / (8 * xt::numeric_constants<value_type>::PI);

  xt::xtensor<value_type, 1, xt::layout_type::row_major> y =
    xt::eval(a*xt::pow(X2 - b * X1*X1 + c * X1 - d, 2) + e * (1 - f)*xt::cos(X1) + e) + 5 * _x1;

  return y;
}

/**
 * @brief Get the bounder of Branin functions
 * 
 * @return std::pair<std::vector<double>, std::vector<double>> 
 */
std::pair<std::vector<double>, std::vector<double>> bounder() const
{
  return { {-5.0, 0.0}, {10.0, 15.0} };
}
};

In [5]:
{
  using xtensor_x_type = xt::xarray<double>;
  using objective_type = Branin;
  using population_type = xevo::Population;
  using selection_type = xevo::Selection_best_pso;
  using velocity_type = xevo::Velocity_ring_topology;
  using position_type = xevo::Position;

  std::array<std::size_t, 2> shape = {40, 2};
  std::array<std::size_t, 1> shape_y = {40};

  xt::xarray<double> X = xt::zeros<double>(shape);
  xt::xarray<double> V = xt::zeros<double>(shape);
  double max_double_value = std::numeric_limits<double>::max();
  
  Branin objective_f;

  xevo::pso pso_algorithm;
  pso_algorithm.initialise<xtensor_x_type,
   population_type, population_type>(X, V);
  
  xt::xarray<double> XB(X);
  xt::xarray<double> YB = xt::ones<double>(shape_y)*max_double_value;

  std::size_t num_generations = 300;
    
  std::size_t no_frames = 10;
  std::size_t skip = static_cast<std::size_t>(std::floor(num_generations / no_frames));
  
  for (auto i{0}; i<num_generations; ++i)
  {
    pso_algorithm.evolve<xtensor_x_type, xtensor_x_type, objective_type,
     position_type, velocity_type, selection_type>(X, XB, YB, V, objective_f, std::make_tuple(),
     std::make_tuple(0.5, 0.8, 0.9), std::make_tuple());
            
    if (i % skip == 0)
    {
      std::string pop_file = "./output/branin_pop_" + std::to_string(i) + ".npy";
      xt::dump_npy(pop_file, X);
    }
  }


std::cout << "Last pop: \n" << X << "\n" << std::endl;
    
}

Last pop: 
{{ 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  0.823907},
 { 0.121579,  